<a href="https://colab.research.google.com/github/MarianaDuartee/ProjetoFinal/blob/main/3_SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Instalando  dependencias

In [1]:
!pip install pyspark
!pip install gcsfs
!pip install pandas


     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 41.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=40cfb3e4a0c1022d16f132d542ff487c5609e3a58315123c5a7fa0d27ccdd6f4
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 1.1 MB 6.5 MB/s 
     |████████████████████████████████| 132 kB 58.0 MB/s 
     |████████████████████████████████| 160 kB 52.5 MB/s 
     |████████████████████████████████| 192 kB 38.3 MB/s 
     |████████████████████████████████| 271 kB 57.2 MB/s 


##Importando dataframes, bibliotecas e abrindo Sparksession

In [2]:
from pyspark.sql import SparkSession
from pyspark import SQLContext
from google.cloud import storage
from pyspark.sql.types import *
from pyspark.sql.types import DateType

import pyspark
import pyspark.sql.functions as F
import os
import gcsfs
import pandas as pd

##Criando uma conexão Spark e configurando chave de serviço



In [6]:
spark = SparkSession.builder\
.master('local')\
.appName('Projeto_Final')\
.config('spark.ui.enable', 'true')\
.config('spark.ui.port', '4050')\
.getOrCreate()

spark

serviceaccount = '/content/soulcode-projeto-final-4b88bea6e07a.json'
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = serviceaccount

## Dataframes exportados do PySpark

In [31]:
# Lista de Dataframes
file_path = ['',
             'gs://data_lake_ingest_data/2_temp/temp_pandas_despesas_normalizado.csv', # df_1
             'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencias_normalizado.json', # df_2
             'gs://data_lake_ingest_data/2_temp/temp_pandas_vitimas_normalizado.json', # df_3
             'gs://data_lake_ingest_data/2_temp/temp_pandas_ocorrencia_vitimas_porAnoEstado.csv', # df_4
             'gs://data_lake_ingest_data/2_temp/temp_pandas_taxa_analfabetismo_normalizado.csv', # df_5
             'gs://data_lake_ingest_data/2_temp/temp_pyspark_agg_1_General_Data.csv', # df_6
             'gs://data_lake_ingest_data/2_temp/temp_pyspark_agg_2_Freq_Escolar_Taxa_Analfa.csv', # df_7
             'gs://data_lake_ingest_data/2_temp/temp_pyspark_agg_3_general_data_2020.csv', # df_8
             'gs://data_lake_ingest_data/2_temp/temp_pyspark_agg_4_freq_escolar_normalizada.csv', # df_9
             'gs://data_lake_ingest_data/2_temp/temp_pyspark_tabela_frequencia_escolar_normalizado.csv'] # df_10

# Lendo dataframes dinâmicamente usando Python, Pandas e SQL
for i in range(len(file_path)):

    fs = gcsfs.GCSFileSystem(project='soulcode-projeto-final', token=serviceaccount)

    if i > 0 and i < 12:
        if ".csv" in file_path[i]: # Lendo csv
            with fs.open(file_path[i]):
                data = pd.read_csv(file_path[i], sep=',', encoding='UTF-8', header=0)

            # Criando visualização temporária SparkSQL
            spark.createDataFrame(data).createOrReplaceTempView('df_'+str(i))
        
        elif ".json" in file_path[i]: # Lendo json
            with fs.open(file_path[i]): 
                data = pd.read_json(file_path[i], encoding='UTF-8')

            # Criando visualização temporária SparkSQL
            spark.createDataFrame(data).createOrReplaceTempView('df_'+str(i))

        elif ".xlsx" in file_path[i]: # Lendo xlsx
            with fs.open(file_path[i]): 
                data = pd.read_excel(file_path[i])

            # Criando visualização temporária SparkSQL
            spark.createDataFrame(data).createOrReplaceTempView('df_'+str(i))

    else:
        continue

# Verificando tipos de dados vindos do PySpark
spark.sql(''' DESCRIBE df_1 ''').show()
spark.sql(''' DESCRIBE df_2 ''').show()
spark.sql(''' DESCRIBE df_3 ''').show()

# Visualizando tabelas
spark.sql('''
        SELECT *
        FROM df_1
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_2
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_3
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_5
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_6
        LIMIT 10
        '''    
        ).show()

spark.sql('''
        SELECT *
        FROM df_7
        LIMIT 10
        '''    
        ).show()

+------------------+---------+-------+
|          col_name|data_type|comment|
+------------------+---------+-------+
|                UF|   string|   null|
|      Despesas2016|   double|   null|
|      Despesas2017|   double|   null|
|      Despesas2018|   double|   null|
|      Despesas2019|   double|   null|
|      Despesas2020|   double|   null|
|         Variacao%|   double|   null|
|Previsao2021|Media|   double|   null|
+------------------+---------+-------+

+-----------+---------+-------+
|   col_name|data_type|comment|
+-----------+---------+-------+
|         UF|   string|   null|
|  TipoCrime|   string|   null|
|        Ano|   bigint|   null|
|        Mes|   string|   null|
|Ocorrencias|   bigint|   null|
+-----------+---------+-------+

+----------+---------+-------+
|  col_name|data_type|comment|
+----------+---------+-------+
|        UF|   string|   null|
| TipoCrime|   string|   null|
|       Ano|   bigint|   null|
|       Mes|   string|   null|
|SexoVitima|   string|   

Separar as planilhas e alterar o arquivo da planilha 6. Acrescentar planilha 7

##Realizando algumas analises no SparkSQL

Total de ocorrencias criminais por Estado (nos anos de 2016 a 2021) Está correto 2021??? De quando foi essa analise?

In [32]:
spark.sql(
'''
SELECT UF, SUM(Ocorrencias) AS Total_Ocorrencias_por_estado
FROM df_2
GROUP BY UF
ORDER BY UF ASC
'''
).show()


+-------------------+----------------------------+
|                 UF|Total_Ocorrencias_por_estado|
+-------------------+----------------------------+
|               Acre|                       10569|
|            Alagoas|                       36240|
|              Amapá|                        9594|
|           Amazonas|                       42023|
|              Bahia|                      159173|
|              Ceará|                      109815|
|   Distrito Federal|                       60852|
|     Espírito Santo|                       71950|
|              Goiás|                      124737|
|           Maranhão|                       59384|
|        Mato Grosso|                       46055|
| Mato Grosso do Sul|                       38831|
|       Minas Gerais|                      216193|
|             Paraná|                      182637|
|            Paraíba|                       23523|
|               Pará|                       94224|
|         Pernambuco|          

Quantidade de vitimas do sexo feminino por tipo de crime

In [33]:
spark.sql(
  
'''
SELECT TipoCrime,
COUNT (Vitimas) AS Feminino
FROM df_3
WHERE SexoVitima = 'Feminino'
GROUP BY TipoCrime
'''

).show()

+--------------------+--------+
|           TipoCrime|Feminino|
+--------------------+--------+
|Roubo seguido de ...|    1380|
|    Homicídio doloso|    1715|
|Lesão corporal se...|    1300|
+--------------------+--------+



Quantidade de vitimas do sexo maculino por tipo de crime

In [34]:
spark.sql(
  
'''
SELECT TipoCrime, 
COUNT (Vitimas) AS Masculino
FROM df_3
WHERE SexoVitima = 'Masculino'
GROUP BY TipoCrime
'''

).show()

+--------------------+---------+
|           TipoCrime|Masculino|
+--------------------+---------+
|Roubo seguido de ...|     1640|
|    Homicídio doloso|     1746|
|Lesão corporal se...|     1513|
+--------------------+---------+



Somando a quantidade de vitimas por sexo e tipo de crime


In [35]:
spark.sql(
  
'''
SELECT TipoCrime, SexoVitima, SUM(Vitimas) AS SomaVitimas
FROM df_3
GROUP BY TipoCrime,SexoVitima
ORDER BY TipoCrime
'''

).show(10)

+--------------------+----------+-----------+
|           TipoCrime|SexoVitima|SomaVitimas|
+--------------------+----------+-----------+
|    Homicídio doloso| Masculino|     228612|
|    Homicídio doloso|   Sexo NI|       8624|
|    Homicídio doloso|  Feminino|      20834|
|Lesão corporal se...|   Sexo NI|        305|
|Lesão corporal se...|  Feminino|        527|
|Lesão corporal se...| Masculino|       3663|
|Roubo seguido de ...| Masculino|       8786|
|Roubo seguido de ...|   Sexo NI|        457|
|Roubo seguido de ...|  Feminino|       1056|
+--------------------+----------+-----------+





Comparando a quantidade de ocorrencias com taxa de analfabetismo




In [36]:
spark.sql(
'''
SELECT  df_4.UF AS Estado, 
        df_4.TipoCrime AS TipoCrime, 
        df_4.Ocorrencias AS Ocorrencias, 
        df_5.TaxaAnalfabetismo2019 AS Anafalbetismo
FROM df_4
INNER JOIN df_5
ON (df_4.UF = df_5.UF)
WHERE df_4.Ano = 2019
'''
).show(truncate=False)

+----------------+-----------------------------------+-----------+-------------+
|Estado          |TipoCrime                          |Ocorrencias|Anafalbetismo|
+----------------+-----------------------------------+-----------+-------------+
|Acre            |Homicídio doloso                   |292        |11.7         |
|Acre            |Lesão corporal seguida de morte    |2          |11.7         |
|Acre            |Roubo seguido de morte (latrocínio)|14         |11.7         |
|Alagoas         |Homicídio doloso                   |1028       |17.1         |
|Alagoas         |Lesão corporal seguida de morte    |6          |17.1         |
|Alagoas         |Roubo seguido de morte (latrocínio)|17         |17.1         |
|Amapá           |Homicídio doloso                   |232        |5.5          |
|Amapá           |Lesão corporal seguida de morte    |23         |5.5          |
|Amapá           |Roubo seguido de morte (latrocínio)|15         |5.5          |
|Amazonas        |Homicídio 

Comparando a quantidade de ocorrencias de 2019 com frequencia escolar com idade acima de 20 anos


In [37]:
spark.sql(
'''
SELECT  df_4.UF AS Estado, 
        df_4.TipoCrime AS TipoCrime, 
        df_4.Ocorrencias AS Ocorrencias, 
        df_10.Media_Freq_25_anos_ou_mais AS Freq_2016a2019
FROM df_4
INNER JOIN df_10
ON (df_4.UF = df_10.UF)
WHERE df_4.Ano = 2019
'''
).show()

+----------------+--------------------+-----------+--------------+
|          Estado|           TipoCrime|Ocorrencias|Freq_2016a2019|
+----------------+--------------------+-----------+--------------+
|            Acre|    Homicídio doloso|        292|           9 %|
|            Acre|Lesão corporal se...|          2|           9 %|
|            Acre|Roubo seguido de ...|         14|           9 %|
|         Alagoas|    Homicídio doloso|       1028|           4 %|
|         Alagoas|Lesão corporal se...|          6|           4 %|
|         Alagoas|Roubo seguido de ...|         17|           4 %|
|           Amapá|    Homicídio doloso|        232|           7 %|
|           Amapá|Lesão corporal se...|         23|           7 %|
|           Amapá|Roubo seguido de ...|         15|           7 %|
|        Amazonas|    Homicídio doloso|        984|           7 %|
|        Amazonas|Lesão corporal se...|         23|           7 %|
|        Amazonas|Roubo seguido de ...|         46|           

Comparando o numero de ocorrencias com valor investido para segurança publica no ano de 2019

In [38]:
spark.sql(
'''
SELECT  df_4.UF AS Estado, 
        df_4.TipoCrime AS TipoCrime, 
        df_4.Ocorrencias AS Ocorrencias, 
        df_1.Despesas2019 AS Investimento
FROM df_1 
INNER JOIN df_4
ON (df_4.UF = df_1.UF)
WHERE df_4.Ano = 2019
'''
).show(10)

+--------+--------------------+-----------+---------------+
|  Estado|           TipoCrime|Ocorrencias|   Investimento|
+--------+--------------------+-----------+---------------+
|    Acre|    Homicídio doloso|        292| 6.9257020034E8|
|    Acre|Lesão corporal se...|          2| 6.9257020034E8|
|    Acre|Roubo seguido de ...|         14| 6.9257020034E8|
| Alagoas|    Homicídio doloso|       1028|1.15915881929E9|
| Alagoas|Lesão corporal se...|          6|1.15915881929E9|
| Alagoas|Roubo seguido de ...|         17|1.15915881929E9|
|   Amapá|    Homicídio doloso|        232| 6.0636632529E8|
|   Amapá|Lesão corporal se...|         23| 6.0636632529E8|
|   Amapá|Roubo seguido de ...|         15| 6.0636632529E8|
|Amazonas|    Homicídio doloso|        984|2.10845995454E9|
+--------+--------------------+-----------+---------------+
only showing top 10 rows



Ranking a taxa de anafalbetismo por estado

In [39]:
spark.sql(
'''
SELECT UF, TaxaAnalfabetismo2019,
RANK() OVER (ORDER BY TaxaAnalfabetismo2019 DESC) AS Rank
FROM df_7
'''
).show(27)


+-------------------+---------------------+----+
|                 UF|TaxaAnalfabetismo2019|Rank|
+-------------------+---------------------+----+
|          Tocantins|                  9 %|   1|
|               Pará|                  8 %|   2|
|        Mato Grosso|                  6 %|   3|
|           Rondônia|                  6 %|   3|
|              Amapá|                  5 %|   5|
|           Amazonas|                  5 %|   5|
|     Espírito Santo|                  5 %|   5|
|              Goiás|                  5 %|   5|
| Mato Grosso do Sul|                  5 %|   5|
|       Minas Gerais|                  5 %|   5|
|            Roraima|                  5 %|   5|
|             Paraná|                  4 %|  12|
|   Distrito Federal|                  2 %|  13|
|  Rio Grande do Sul|                  2 %|  13|
|     Rio de Janeiro|                  2 %|  13|
|     Santa Catarina|                  2 %|  13|
|          São Paulo|                  2 %|  13|
|            Alagoas